<a href="https://colab.research.google.com/github/abdoulayegk/Zindi-Hackathons/blob/main/Layer_ai_Air_Quality_Prediction_Challenge_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Package to have access to the Zindi Platform features
!pip -q install git+https://github.com/eaedk/testing-zindi-package.git

In [ ]:
from zindi.user import Zindian

#@title Input Username

# Login info for a Zindi Account

USERNAME = "Abdoulayegk" #@param {type : "string"}

# object
user = Zindian(username=USERNAME) # Si

In [ ]:
user.select_a_challenge(reward='all', kind='competition', active='true')

In [ ]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

In [ ]:
!pip install -q catboost
!pip install -q optuna


In [ ]:
import matplotlib.pyplot as plt
import missingno as mn
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold, TimeSeriesSplit, RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, ShuffleSplit, TimeSeriesSplit,RepeatedKFold
from xgboost import XGBRegressor 
import warnings
warnings.filterwarnings('ignore')
from random import seed
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer

seed = 42
seed = seed
np.random.seed(seed)
pd.set_option("display.max_columns", 68)

In [ ]:
train = pd.read_csv('dataset/train.csv').drop(['ID','device'],axis=1)
test = pd.read_csv('dataset/test.csv').drop(['ID','device'],axis=1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

I am going to look at the correlation between features I am going to do it base on the target

In [ ]:
train.corr()['pm2_5'].sample(60).sort_values(ascending=False)

In [ ]:
# Plot missing values in train set
ax = train.isna().sum().sort_values().plot(kind = 'barh', figsize = (16, 13))
plt.title('Percentage of Missing Values Per Column in Train Set', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/train.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))


In [ ]:
train.loc[:, 'site_latitude':'pm2_5'].describe().T.style.bar(subset=['mean'], color='#206ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

### Distrubition of the features on the training and test data

In [ ]:
features = train.columns.values[3:13]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.distplot(train[feature], hist=False,label='train')
    sns.distplot(test[feature], hist=False,label='test')
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

In [ ]:
features = train.columns.values[13:23]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.distplot(train[feature], hist=False,label='train')
    sns.distplot(test[feature], hist=False,label='test')
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

In [ ]:
features = train.columns.values[23:33]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.distplot(train[feature], hist=False,label='train')
    sns.distplot(test[feature], hist=False,label='test')
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

* we can see that most of the features are skewed either on the right or left so to solve this problem we will try to do the following:<br>
    1. Log transformation of the features that are skewed <br>
    2. Sqrt of the features that are skewed 

### Boxplot for different features to detect outliers.

In [ ]:
eatures = train.columns.values[13:23]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.boxplot(train[feature])
    sns.boxplot(test[feature])
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

In [ ]:
eatures = train.columns.values[13:23]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.boxplot(train[feature])
    #sns.boxplot(test[feature])
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

In [ ]:
eatures = train.columns.values[13:23]
i = 0
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(10,10,figsize=(18,22))

for feature in features:
    i += 1
    plt.subplot(5,2,i)
    sns.boxplot(train[feature])
    #sns.boxplot(test[feature])
    plt.xlabel(feature, fontsize=9)
    locs, labels = plt.xticks()
    plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
    plt.tick_params(axis='y', which='major', labelsize=6)
plt.show();

## Features engineering

In [ ]:
# Extract day, month year and hour from the date column
# day
def converte_dates(df):
    
    df['date'] = pd.to_datetime(df['date'])
    
    #
    df['date_day'] = df.date.dt.day

    # month
    df['date_month'] = df.date.dt.month

    # year
    df['date_year'] = df.date.dt.year

    # hour
    df['date_hour'] = df.date.dt.hour
    
    # minute
    df['date_minute'] = df.date.dt.minute
    
    # day of week
    df['date_dayofweek'] = df.date.dt.weekday
    
    
    return df


train = converte_dates(train).drop('date', axis=1)
test = converte_dates(test).drop('date', axis=1)

Here I am capturing NaN per row and making new feature.
I am doing this because sometime missing carry signal so we only give it to the model.

In [ ]:
def feature_engineering(df):
    df['NaN_row'] = df.isna().sum(axis=1)
    df['std'] = df.std(axis=1)
    return df

train = feature_engineering(train)
test = feature_engineering(test)

In [ ]:
### Log transform for some features


In [ ]:
# Features selection

X = train.drop('pm2_5',axis=1)
y = train.pm2_5

In [ ]:
X.shape, test.shape

In [ ]:
pipeline = Pipeline([
    ('impute', IterativeImputer(random_state=seed)),
    ('scale', RobustScaler())
    #('quantiletransform', QuantileTransformer(random_state=seed))
])

X = pd.DataFrame(columns=X.columns, data=pipeline.fit_transform(X))
test = pd.DataFrame(columns=test.columns, data=pipeline.transform(test))

In [ ]:
X.head()

In [ ]:
fold_pred=[]
oof_pred = []

params = {'colsample_bytree': 0.384606276881856,
 'learning_rate': 0.23251453473545997,
 'max_depth': 7,
 'subsample': 0.3530887571489526}




fold= KFold(n_splits=5)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):     
  
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = np.log1p(y.iloc[train_index]), y.iloc[test_index]

    model = LGBMRegressor(**params, objective = "mae")
    model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=300, verbose = False)#erly100

    preds= model.predict(X_test)
    print("err: ",(mean_absolute_error(y_test,np.expm1(preds))))  #Reverse transformation
    oof_pred.append(mean_absolute_error(y_test,np.expm1(preds)))
    p2 = model.predict(test[X.columns])
    fold_pred.append(np.expm1(p2))
    

print(np.mean(oof_pred))

## Catboost

In [ ]:
err1=[]
y_pred_totcb1=[]

fold=KFold(n_splits=10, random_state=seed, shuffle=True)
for train_index, test_index in fold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2 = CatBoostRegressor(task_type='GPU',loss_function='RMSE',)
    m2.fit(X_train,y_train,eval_set=[(X_test, y_test)], verbose=0,)
    preds = m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    err1.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test)
    y_pred_totcb1.append(p2)
np.mean(err1)

# LightGBM

In [ ]:
# err1=[]
# y_pred_totcb1=[]

# fold=KFold(n_splits=10, random_state=seed,shuffle=True)
# for train_index, test_index in fold.split(X, y):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     m2 = LGBMRegressor()
#     m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], verbose=100,)
#     preds = m2.predict(X_test)
#     print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
#     err1.append(np.sqrt(mean_squared_error(y_test,preds)))
#     p2 = m2.predict(test)
#     y_pred_totcb1.append(p2)
# np.mean(err1)

## XGBoost

In [ ]:
# err1=[]
# y_pred_totcb1=[]

# fold=KFold(n_splits=10, random_state=seed,shuffle=True)
# for train_index, test_index in fold.split(X, y):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     m2 = XGBRegressor()
#     m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], verbose=100,)
#     preds = m2.predict(X_test)
#     print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
#     err1.append(np.sqrt(mean_squared_error(y_test,preds)))
#     p2 = m2.predict(test)
#     y_pred_totcb1.append(p2)
# np.mean(err1)

In [ ]:
ss = pd.read_csv('dataset/SampleSubmission.csv')
ss.head()

In [ ]:
predic1 = np.mean(y_pred_totcb1, 0)
a = {"pm2_5": predic1}
predic1

In [ ]:
sub = {'ID':ss.ID,'pm2_5':predic1}
s = pd.DataFrame(sub)
s.to_csv('catboost.csv',index=False)

In [ ]:
# # 0.7831404345536787
feature_importance_df = pd.DataFrame(m2.feature_importances_, columns=['importance'])
feature_importance_df['feature'] = X.columns

plt.figure(figsize=(20, 12));
sns.barplot(x="importance", y="feature", data=feature_importance_df.sort_values(by = ['importance'], ascending = False).head(60))
plt.title('CatboostClassifier features importance (top 50):')